In [9]:
import numpy as np
from scipy.interpolate import interp1d
from PIL import Image
img_path = 'sunlit_loung.jpg'
img_pil = Image.open(img_path)

width, height = img_pil.size
print(width)
print(height)
edited_rgb_vals = []

def rgb_to_wavelength(rgb):
    rgb = np.array(rgb) / 255.0

    if np.all(rgb == 0):  # Black
        return 0
    if np.all(rgb == 1):  # White
        return 570

    # Corresponding coefficients to convert RGB to XYZ
    rgb_to_xyz_matrix = np.array([
        [0.4124564, 0.3575761, 0.1804375],
        [0.2126729, 0.7151522, 0.0721750],
        [0.0193339, 0.1191920, 0.9503041]
    ])

    mask = rgb > 0.04045
    rgb[mask] = ((rgb[mask] + 0.055) / 1.055) ** 2.4
    rgb[~mask] /= 12.92

    xyz = np.dot(rgb_to_xyz_matrix, rgb)

    # X, Y chromaticity values
    sum_xyz = np.sum(xyz)
    if sum_xyz == 0:
        return None  # Black
    x, y = xyz[0] / sum_xyz, xyz[1] / sum_xyz

    # Spectral locus data (x, y, wavelength)
    spectral_locus = np.array([
        [0.1741, 0.0050, 380], [0.1741, 0.0050, 381], [0.1740, 0.0050, 382],
    [0.1740, 0.0050, 383], [0.1739, 0.0049, 384], [0.1739, 0.0049, 385],
    [0.1738, 0.0049, 386], [0.1738, 0.0049, 387], [0.1737, 0.0049, 388],
    [0.1737, 0.0049, 389], [0.1736, 0.0049, 390], [0.1736, 0.0049, 391],
    [0.1735, 0.0049, 392], [0.1735, 0.0049, 393], [0.1734, 0.0048, 394],
    [0.1734, 0.0048, 395], [0.1733, 0.0048, 396], [0.1733, 0.0048, 397],
    [0.1732, 0.0048, 398], [0.1732, 0.0048, 399], [0.1731, 0.0048, 400],
    [0.1730, 0.0048, 401], [0.1729, 0.0048, 402], [0.1728, 0.0048, 403],
    [0.1727, 0.0048, 404], [0.1726, 0.0048, 405], [0.1725, 0.0048, 406],
    [0.1724, 0.0048, 407], [0.1723, 0.0048, 408], [0.1722, 0.0048, 409],
    [0.1721, 0.0048, 410], [0.1720, 0.0048, 411], [0.1719, 0.0048, 412],
    [0.1717, 0.0049, 413], [0.1716, 0.0049, 414], [0.1714, 0.0050, 415],
    [0.1712, 0.0051, 416], [0.1710, 0.0052, 417], [0.1708, 0.0053, 418],
    [0.1706, 0.0054, 419], [0.1703, 0.0055, 420], [0.1701, 0.0057, 421],
    [0.1698, 0.0059, 422], [0.1695, 0.0061, 423], [0.1692, 0.0063, 424],
    [0.1689, 0.0066, 425], [0.1685, 0.0069, 426], [0.1681, 0.0072, 427],
    [0.1677, 0.0076, 428], [0.1673, 0.0079, 429], [0.1669, 0.0083, 430],
    [0.1664, 0.0087, 431], [0.1660, 0.0091, 432], [0.1655, 0.0095, 433],
    [0.1650, 0.0100, 434], [0.1644, 0.0105, 435], [0.1639, 0.0110, 436],
    [0.1633, 0.0115, 437], [0.1627, 0.0121, 438], [0.1621, 0.0126, 439],
    [0.1614, 0.0132, 440], [0.1607, 0.0138, 441], [0.1600, 0.0145, 442],
    [0.1592, 0.0152, 443], [0.1584, 0.0159, 444], [0.1576, 0.0167, 445],
    [0.1567, 0.0175, 446], [0.1558, 0.0183, 447], [0.1549, 0.0192, 448],
    [0.1539, 0.0201, 449], [0.1529, 0.0211, 450], [0.1518, 0.0221, 451],
    [0.1507, 0.0232, 452], [0.1495, 0.0243, 453], [0.1483, 0.0255, 454],
    [0.1470, 0.0267, 455], [0.1457, 0.0280, 456], [0.1443, 0.0294, 457],
    [0.1429, 0.0308, 458], [0.1414, 0.0323, 459], [0.1399, 0.0339, 460],
    [0.1383, 0.0356, 461], [0.1367, 0.0374, 462], [0.1350, 0.0392, 463],
    [0.1333, 0.0412, 464], [0.1315, 0.0433, 465], [0.1296, 0.0455, 466],
    [0.1277, 0.0478, 467], [0.1257, 0.0502, 468], [0.1237, 0.0528, 469],
    [0.1216, 0.0555, 470], [0.1194, 0.0583, 471], [0.1172, 0.0613, 472],
    [0.1149, 0.0645, 473], [0.1126, 0.0678, 474], [0.1102, 0.0714, 475],
    [0.1078, 0.0751, 476], [0.1053, 0.0790, 477], [0.1027, 0.0832, 478],
    [0.1000, 0.0876, 479], [0.0973, 0.0923, 480], [0.0945, 0.0973, 481],
    [0.0917, 0.1025, 482], [0.0888, 0.1081, 483], [0.0858, 0.1140, 484],
    [0.0828, 0.1202, 485], [0.0797, 0.1268, 486], [0.0766, 0.1337, 487],
    [0.0734, 0.1410, 488], [0.0702, 0.1486, 489], [0.0669, 0.1567, 490],
    [0.0636, 0.1652, 491], [0.0603, 0.1741, 492], [0.0570, 0.1835, 493],
    [0.0537, 0.1933, 494], [0.0504, 0.2036, 495], [0.0471, 0.2143, 496],
    [0.0438, 0.2255, 497], [0.0406, 0.2372, 498], [0.0374, 0.2493, 499],
    [0.0343, 0.2618, 500], [0.0312, 0.2747, 501], [0.0283, 0.2880, 502],
    [0.0254, 0.3016, 503], [0.0227, 0.3156, 504], [0.0200, 0.3299, 505],
    [0.0175, 0.3444, 506], [0.0151, 0.3592, 507], [0.0129, 0.3742, 508],
    [0.0108, 0.3893, 509], [0.0089, 0.4046, 510], [0.0072, 0.4200, 511],
    [0.0057, 0.4354, 512], [0.0045, 0.4508, 513], [0.0035, 0.4661, 514],
    [0.0028, 0.4812, 515], [0.0024, 0.4961, 516], [0.0023, 0.5107, 517],
    [0.0026, 0.5250, 518], [0.0032, 0.5389, 519], [0.0042, 0.5524, 520],
    [0.0056, 0.5654, 521], [0.0074, 0.5779, 522], [0.0097, 0.5899, 523],
    [0.0125, 0.6014, 524], [0.0158, 0.6123, 525], [0.0196, 0.6226, 526],
    [0.0240, 0.6323, 527], [0.0290, 0.6414, 528], [0.0346, 0.6499, 529],
    [0.0408, 0.6577, 530], [0.0477, 0.6648, 531], [0.0552, 0.6712, 532],
    [0.0634, 0.6769, 533], [0.0723, 0.6819, 534], [0.0819, 0.6861, 535],
    [0.0922, 0.6896, 536], [0.1032, 0.6923, 537], [0.1149, 0.6942, 538],
    [0.1273, 0.6953, 539], [0.1404, 0.6956, 540], [0.1541, 0.6951, 541],
    [0.1685, 0.6938, 542], [0.1835, 0.6917, 543], [0.1991, 0.6888, 544],
    [0.2152, 0.6852, 545], [0.2318, 0.6807, 546], [0.2489, 0.6755, 547],
    [0.2664, 0.6695, 548], [0.2843, 0.6627, 549], [0.3026, 0.6552, 550],
    [0.3211, 0.6470, 551], [0.3399, 0.6381, 552], [0.3589, 0.6285, 553],
    [0.3780, 0.6183, 554], [0.3972, 0.6075, 555], [0.4164, 0.5962, 556],
    [0.4356, 0.5844, 557], [0.4547, 0.5721, 558], [0.4737, 0.5594, 559],
    [0.4926, 0.5463, 560], [0.5112, 0.5329, 561], [0.5296, 0.5193, 562],
    [0.5477, 0.5054, 563], [0.5654, 0.4914, 564], [0.5828, 0.4773, 565],
    [0.5998, 0.4631, 566], [0.6163, 0.4489, 567], [0.6324, 0.4347, 568],
    [0.6480, 0.4206, 569], [0.6631, 0.4066, 570], [0.6777, 0.3928, 571],
    [0.6917, 0.3792, 572], [0.7052, 0.3659, 573], [0.7181, 0.3529, 574],
    [0.7305, 0.3402, 575], [0.7423, 0.3279, 576], [0.7535, 0.3160, 577],
    [0.7642, 0.3045, 578], [0.7743, 0.2935, 579], [0.7838, 0.2829, 580],
    [0.7927, 0.2729, 581], [0.8011, 0.2633, 582], [0.8089, 0.2543, 583],
    [0.8162, 0.2458, 584], [0.8229, 0.2378, 585], [0.8291, 0.2303, 586],
    [0.8348, 0.2233, 587], [0.8401, 0.2168, 588], [0.8448, 0.2108, 589],
    [0.8491, 0.2052, 590], [0.8530, 0.2000, 591], [0.8565, 0.1953, 592],
    [0.8596, 0.1909, 593], [0.8624, 0.1869, 594], [0.8649, 0.1833, 595],
    [0.8671, 0.1800, 596], [0.8690, 0.1770, 597], [0.8706, 0.1744, 598],
    [0.8720, 0.1720, 599], [0.8733, 0.1699, 600], [0.8743, 0.1680, 601],
    [0.8751, 0.1664, 602], [0.8758, 0.1650, 603], [0.8764, 0.1638, 604],
    [0.8769, 0.1628, 605], [0.8773, 0.1619, 606], [0.8776, 0.1611, 607],
    [0.8778, 0.1605, 608], [0.8780, 0.1600, 609], [0.8781, 0.1596, 610],
    [0.8782, 0.1593, 611], [0.8782, 0.1590, 612], [0.8782, 0.1588, 613],
    [0.8782, 0.1586, 614], [0.8782, 0.1585, 615], [0.8782, 0.1584, 616],
    [0.8781, 0.1583, 617], [0.8781, 0.1582, 618], [0.8780, 0.1582, 619],
    [0.8779, 0.1581, 620], [0.8779, 0.1581, 621], [0.8778, 0.1581, 622],
    [0.8777, 0.1581, 623], [0.8776, 0.1581, 624], [0.8775, 0.1581, 625],
    [0.8774, 0.1581, 626], [0.8773, 0.1581, 627], [0.8772, 0.1581, 628],
    [0.8771, 0.1581, 629], [0.8770, 0.1582, 630], [0.8769, 0.1582, 631]])

    # Find the closest point on the spectral locus
    distances = np.sqrt((spectral_locus[:, 0] - x)**2 + (spectral_locus[:, 1] - y)**2)
    closest_index = np.argmin(distances)

    # Return the wavelength of the closest point without interpolation
    return spectral_locus[closest_index, 2]

def cat_wavelength_to_rgb(cat_wavelength_val):
  def blue_cone_response(w):
        return np.exp(-((w - 450)**2) / (2 * 55**2))   # Gaussian function centered at 450 with SD of 55

  def green_cone_response(w):
        return np.exp(-((w - 550)**2) / (2 * 55**2))  # Gaussian function centered at 550 with SD of 55

  blue_response = blue_cone_response(cat_wavelength_val)
  green_response = green_cone_response(cat_wavelength_val)


  total_response = blue_response + green_response
  if total_response > 0:
      blue_response /= total_response
      green_response /= total_response

  r = int(green_response * 255)
  g = int(green_response * 255)
  b = int(blue_response * 255)

  return (r, g, b)

for i in range(width):
  for j in range(height):
    r, g, b = img_pil.getpixel((i, j))
    print("r", r)
    print("g", g)
    print("b", b)
    wavelength = rgb_to_wavelength((r, g, b))
    print("i", i)
    print("j", j)
    print("wavelength", wavelength)
    cat_r, cat_g, cat_b = cat_wavelength_to_rgb(wavelength)
    print("cat_r", cat_r)
    print("cat_g", cat_g)
    print("cat_b", cat_b)
    print("-------------------------")
    edited_rgb_vals.append((cat_r, cat_g, cat_b))

img_pil.putdata(edited_rgb_vals)
img_pil.save('edited_image.jpg')




Streaming output truncated to the last 5000 lines.
r 104
g 94
b 105
i 222
j 175
wavelength 497.0
cat_r 121
cat_g 121
cat_b 133
-------------------------
r 71
g 63
b 60
i 222
j 176
wavelength 560.0
cat_r 224
cat_g 224
cat_b 30
-------------------------
r 70
g 63
b 57
i 222
j 177
wavelength 559.0
cat_r 223
cat_g 223
cat_b 31
-------------------------
r 67
g 61
b 61
i 222
j 178
wavelength 559.0
cat_r 223
cat_g 223
cat_b 31
-------------------------
r 68
g 61
b 68
i 222
j 179
wavelength 498.0
cat_r 123
cat_g 123
cat_b 131
-------------------------
r 71
g 65
b 77
i 222
j 180
wavelength 496.0
cat_r 119
cat_g 119
cat_b 135
-------------------------
r 75
g 71
b 86
i 222
j 181
wavelength 496.0
cat_r 119
cat_g 119
cat_b 135
-------------------------
r 80
g 78
b 91
i 222
j 182
wavelength 497.0
cat_r 121
cat_g 121
cat_b 133
-------------------------
r 83
g 83
b 91
i 222
j 183
wavelength 498.0
cat_r 123
cat_g 123
cat_b 131
-------------------------
r 76
g 77
b 79
i 222
j 184
wavelength 500.0
cat_r 

In [ ]:
import numpy as np

def converting_rgb_to_xyz(rgb):
  normalized_rgb = np.array(rgb) / 255.0

  mask = normalized_rgb > 0.04045
  normalized_rgb[mask] = ((normalized_rgb[mask] + 0.055) / 1.055) ** 2.4
  normalized_rgb[~mask] = normalized_rgb[~mask] / 12.92

  normalized_rgb *= 100

  x =  0.4124 * normalized_rgb[0] + 0.3576 * normalized_rgb[1] + 0.1805 * normalized_rgb[2]
  y =  0.2126 * normalized_rgb[0] + 0.7152 * normalized_rgb[1] + 0.0722 * normalized_rgb[2]
  z =  0.0193 * normalized_rgb[0] + 0.1192 * normalized_rgb[1] + 0.9505 * normalized_rgb[2]

  return np.array([x, y, z])

def xyz_to_xy(xyz):
  if np.sum(xyz) == 0.0:
    return np.array([0.0, 0.0])
  return xyz[:2] / np.sum(xyz)

def xy_to_wavelength(xyz_sum):
  xy_wavelength_map = [
    ((0.1741, 0.0050), 380), ((0.3515, 0.0016), 450),
    ((0.3, 0.6), 500), ((0.13902, 0.0813), 550),
    ((0.175, 0.395), 600), ((0.4, 0.5), 650),
    ((0.7347, 0.2653), 700), ((0.7347, 0.2653), 780)
  ]

  closest = min(xy_wavelength_map, key=lambda p: np.linalg.norm(np.array(p[0]) - xyz_sum))
  return closest[1]

def rgb_to_wavelength(rgb):
  xyz = converting_rgb_to_xyz(rgb)
  xy = xyz_to_xy(xyz)
  return xy_to_wavelength(xy)

rgb = (42, 27, 58)
wavelength = rgb_to_wavelength(rgb)
print(wavelength)



550


In [ ]:
def cat_wavelength_to_rgb(cat_wavelength_val):
  def blue_cone_response(w):
        return np.exp(-((w - 450)**2) / (2 * 30**2))

  def green_cone_response(w):
        return np.exp(-((w - 550)**2) / (2 * 30**2))

  # Calculate cone responses
  blue_response = blue_cone_response(cat_wavelength_val)
  green_response = green_cone_response(cat_wavelength_val)

  # Normalize responses
  total_response = blue_response + green_response
  if total_response > 0:
      blue_response /= total_response
      green_response /= total_response

  # Convert to RGB
  # Cats don't see red, so we'll use green for the R channel
  r = int(green_response * 255)
  g = int(green_response * 255)
  b = int(blue_response * 255)

  return (r, g, b)

print(cat_wavelength_to_rgb(490))

(63, 63, 191)


In [4]:
from PIL import Image

im = Image.open("sunlit_lounge.png")
rgb_im = im.convert('RGB')
rgb_im.save('sunlit_loung.jpg')